In [1]:
import os

In [2]:
%pwd

'd:\\waterQualityPrediction\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path

In [6]:
from waterQualityPrediction.constants import *
from waterQualityPrediction.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForest

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
        )

        return model_evaluation_config

In [8]:
from waterQualityPrediction import logger
import joblib
from sklearn.ensemble import RandomForestClassifier
import os
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

C:\Users\Ali\AppData\Local\Temp\ipykernel_6684\28979713.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def classification_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        class_report = classification_report(actual, pred)
        return accuracy, class_report
    
    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.iloc[:, :-1]
        test_y = test_data.iloc[:, -1]   

        
        predicted_qualities = model.predict(test_x)

        (accuracy, class_report) = self.classification_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"accuracy": accuracy, "classification-report": class_report}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-01-28 12:10:13,383: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-28 12:10:13,386: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-28 12:10:13,387: INFO: common: created directory at: artifacts]
[2024-01-28 12:10:13,389: INFO: common: created directory at: artifacts/model_evaluation]


[2024-01-28 12:10:13,898: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
